# Feature postprocessing from paper.
- Title:  "Predicting yeast synthetic lethal genetic interactions using protein domains"
- Authors: Bo Li, Feng Luo,School of Computing,Clemson University,Clemson, SC, USA
- e-mail: bol, luofeng@clemson.edu
- year:2009

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict 
import seaborn as sns
import matplotlib.cm as cm
import scipy as scipy
import random

### Importing datasets 

#### Link to the github repo where the datasets to be downloaded:
[DOWNLOAD THE DATASETS HERE](https://github.com/leilaicruz/machine-learning-for-yeast/tree/dev_Leila/datasets-for-learning)

In [2]:
import os
script_dir = os.path.dirname('__file__') #<-- absolute dir the script is in
rel_path_SL = "datasets/data-synthetic-lethals.xlsx"
rel_path_nSL="datasets/data-positive-genetic.xlsx"
rel_path_domains="datasets/proteins-domains-from-Pfam.xlsx"

abs_file_path_SL = os.path.join(script_dir, rel_path_SL)
abs_file_path_nSL = os.path.join(script_dir, rel_path_nSL)
abs_file_path_domains = os.path.join(script_dir, rel_path_domains)

# os.chdir('mini_book/docs/') #<-- for binder os.chdir('../')
# os.chdir('../')
my_path_sl= abs_file_path_SL
my_path_non_sl=abs_file_path_nSL
my_path_domains=abs_file_path_domains

data_sl=pd.read_excel(my_path_sl,header=0)
data_domains=pd.read_excel(my_path_domains,header=0,index_col='Unnamed: 0')
data_domains=data_domains.dropna()
data_nonsl=pd.read_excel(my_path_non_sl,header=0)

## Building the feature matrix
One matrix for true SL where each row is one pair of SL. Every raw will be a vector of 0,1 or 2 depending on the comparison with the domain list. For row i the jth element = 0 if the jth element of the domain list is not in neither protein A and B, 1, if it is in one of them and 2 if it is in both of them .

### Building the list of proteins domains id per protein pair separately :
- List of protein A: Search for the Sl/nSL database the query gene name and look in the protein domain database which protein domains id has each of those queries.
- List of protein B: Search for the Sl/nSL database the target gene name of the previous query and look in the protein domain database which protein domains id has each of those target genes.

In [3]:
# Selecting the meaningful columns in the respective dataset
domain_id_list=data_domains['domain-name']
query_gene=data_sl['gene-query-name']
target_gene=data_sl['gene-target-name']
query_gene_nonlethal=data_nonsl['gene-query-name']
target_gene_nonlethal=data_nonsl['gene-target-name']



# Initialising the arrays
protein_a_list=[]
protein_b_list=[]
protein_a_list_non=[]
protein_b_list_non=[]

population = np.arange(0,len(data_sl))

# For loop for 10000 pairs sampled randomly from the SL/nSl pair list , and creating a big array of proteind domains id per protein pair

for m in random.sample(list(population), 500):
    protein_a=data_domains[data_domains['name']==query_gene[m]]
    protein_b=data_domains[data_domains['name']==target_gene[m]]
    protein_a_list.append(protein_a['domain-name'].tolist())
    protein_b_list.append(protein_b['domain-name'].tolist())

    protein_a_non=data_domains[data_domains['name']==query_gene_nonlethal[m]]
    protein_b_non=data_domains[data_domains['name']==target_gene_nonlethal[m]]
    protein_a_list_non.append(protein_a_non['domain-name'].tolist())
    protein_b_list_non.append(protein_b_non['domain-name'].tolist())

In [4]:
print('We are going to analyze',len((protein_a_list)) ,'protein pairs, out of',len(data_sl),'SL protein pairs')
print('We are going to analyze',len((protein_a_list_non)) ,'protein pairs, out of',len(data_nonsl),'positive protein pairs')

We are going to analyze 500 protein pairs, out of 17871 SL protein pairs
We are going to analyze 500 protein pairs, out of 43340 positive protein pairs


### Postprocessing #1:  Remove protein pairs from study if either protein in the pair does not contain any domain

In [5]:
def remove_empty_domains(protein_list_search,protein_list_pair):
    index=[]
    for i in np.arange(0,len(protein_list_search)):
        if protein_list_search[i]==[] or protein_list_pair[i]==[]:
            index.append(i) ## index of empty values for the protein_a_list meaning they dont have any annotated domain

    y=[x for x in np.arange(0,len(protein_list_search)) if x not in index] # a list with non empty values from protein_a list

    protein_list_search_new=[]
    protein_list_pair_new=[]
    for i in y:
        protein_list_search_new.append(protein_list_search[i])
        protein_list_pair_new.append(protein_list_pair[i])
    return protein_list_search_new,protein_list_pair_new

## evaluating the function

protein_a_list_new,protein_b_list_new=remove_empty_domains(protein_a_list,protein_b_list)

protein_a_list_non_new,protein_b_list_non_new=remove_empty_domains(protein_a_list_non,protein_b_list_non)

In [6]:
print('The empty domain in the SL were:', len(protein_a_list)-len(protein_a_list_new), 'out of', len(protein_a_list),'domains')
print('The empty domain in the nSL were:', len(protein_a_list_non)-len(protein_a_list_non_new), 'out of', len(protein_a_list_non),'domains')

The empty domain in the SL were: 90 out of 500 domains
The empty domain in the nSL were: 120 out of 500 domains


### Feature engineering: Select from each ordered indexes of domain id list which of them appear once, in both or in any of the domains of each protein pair

#### Define function `get_indexes`

In [7]:
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y] # a function that give the index of whether a value appear in array or not
a=[1,2,2,4,5,6,7,8,9,10]
get_indexes(2,a)

[1, 2]

In [8]:
def feature_building(protein_a_list_new,protein_b_list_new):
    x = np.unique(domain_id_list)
    ## To avoid taking repeated domains from one protein of the pairs , lets reduced the domains of each protein from the pairs to their unique members
    protein_a_list_unique=[]
    protein_b_list_unique=[]
    for i in np.arange(0,len(protein_a_list_new)):
        protein_a_list_unique.append(np.unique(protein_a_list_new[i]))
        protein_b_list_unique.append(np.unique(protein_b_list_new[i]))
        
    protein_feat_true=np.zeros(shape=(len(x),len(protein_a_list_unique)))
    pair_a_b_array=[]
    for i in np.arange(0,len(protein_a_list_unique)):
        index_a=[]
        
        pair=[protein_a_list_unique[i],protein_b_list_unique[i]]
        pair_a_b=np.concatenate(pair).ravel()
        pair_a_b_array.append(pair_a_b)

    j=0
    for i in pair_a_b_array:  
        array,index,counts=np.unique(i,return_index=True,return_counts=True)
        
        for k,m in zip(counts,array):
            if k ==2:
                protein_feat_true[get_indexes(m,x),j]=2
                
            if k==1:
                protein_feat_true[get_indexes(m,x),j]=1
        j=j+1
    return protein_feat_true

In [9]:
protein_feat_true=feature_building(protein_b_list_new=protein_b_list_new,protein_a_list_new=protein_a_list_new)
protein_feat_true_pd=pd.DataFrame(protein_feat_true.T)

protein_feat_non_true=feature_building(protein_b_list_new=protein_b_list_non_new,protein_a_list_new=protein_a_list_non_new)
protein_feat_non_true_pd=pd.DataFrame(protein_feat_non_true.T)

### How many ones and twos are in each dataset

In [10]:
index_2_true=protein_feat_true_pd.where(protein_feat_true_pd==2)
index_2_true_count=index_2_true.count(axis=1).sum()

index_1_true=protein_feat_true_pd.where(protein_feat_true_pd==1)
index_1_true_count=index_1_true.count(axis=1).sum()

index_2_nontrue=protein_feat_non_true_pd.where(protein_feat_non_true_pd==2)
index_2_nontrue_count=index_2_nontrue.count(axis=1).sum()

index_1_nontrue=protein_feat_non_true_pd.where(protein_feat_non_true_pd==1)
index_1_nontrue_count=index_1_nontrue.count(axis=1).sum()


print('fraction of twos in the SL array is',index_2_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)))
print('fraction of ones in the SL array is',index_1_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)))
print('fraction of twos in the PI array is',index_2_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)))
print('fraction of ones in the PI array is',index_1_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)))

fraction of twos in the SL array is 2.4188671638782503e-05
fraction of ones in the SL array is 0.0010086676073372304
fraction of twos in the PI array is 8.69943453675511e-06
fraction of ones in the PI array is 0.0009056111352762071


#### Bar plot to visualize these numbers

In [11]:
plt.bar(['fraction of 2 in the nSL','fraction of 1 in the nSL'],[index_2_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)),index_1_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns))],alpha=0.6,color=['blue','lightblue']), 

plt.bar(['fraction of 2 in SL ','fraction of 1 in SL'],[index_2_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)),index_1_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns))],alpha=0.6,color=['coral','lightcoral'])

plt.ylabel('Fraction from the population')
plt.yscale('log')
plt.xticks(rotation=40)

([0, 1, 2, 3],
 [Text(0, 0, ''), Text(0, 0, ''), Text(0, 0, ''), Text(0, 0, '')])

#### Adding the labels(response variables) to each dataset

In [12]:
protein_feat_true_pd['lethality']=np.ones(shape=(len(protein_a_list_new)))
protein_feat_non_true_pd['lethality']=np.zeros(shape=(len(protein_a_list_non_new)))

#### Joining both datasets

In [13]:
feature_post=pd.concat([protein_feat_true_pd,protein_feat_non_true_pd],axis=0)
feature_post=feature_post.set_index(np.arange(0,len(protein_a_list_new)+len(protein_a_list_non_new)))
print('The number of features are:',feature_post.shape[1])
print('The number of samples are:',feature_post.shape[0])

The number of features are: 3026
The number of samples are: 790


### Postprocessing and exploration of the feature matrix of both datasets

In [14]:
mean=feature_post.T.describe().loc['mean']
std=feature_post.T.describe().loc['std']
lethality=feature_post['lethality']

corr_keys=pd.concat([mean,std,lethality],axis=1)

### Viz of the stats

In [15]:
pair=sns.pairplot(corr_keys,hue='lethality',diag_kind='kde',kind='reg',palette='colorblind')
pair.fig.suptitle('Pairplot to see data dependencies with Lethality',y=1.08)

Text(0.5, 1.08, 'Pairplot to see data dependencies with Lethality')

In [16]:
X, y = feature_post.drop(columns=["lethality"]), feature_post["lethality"]

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_sl=protein_feat_true_pd.drop(columns=['lethality'])
x_nsl=protein_feat_non_true_pd.drop(columns=['lethality'])

X_sl_scaled = scaler.fit_transform(x_sl)
X_nsl_scaled = scaler.fit_transform(x_nsl)

In [18]:
np.shape(X_sl_scaled)

(410, 3025)

## How redundant are each of the protein domains?

In [19]:
def PCA_component_contribution(scaled_matrix,original_data):
    from sklearn.decomposition import PCA

    model = PCA(0.95).fit(scaled_matrix)
    ## apply dimensionality reduction to X_train
    output_pca = model.transform(scaled_matrix)
    
    total=sum(model.explained_variance_)


   
    # number of components , that it will be the number of main axes times the number of original components 
    n_pcs= model.components_.shape[0] # the amount of non redundant protein domains 

    # get the index of the most important feature on EACH component
    # LIST COMPREHENSION HERE
    most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

    initial_feature_names = original_data.columns
    # get the names
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
    

    # LIST COMPREHENSION HERE AGAIN
    dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

    # build the dataframe
    df = pd.DataFrame(dic.items(),columns=['pca-component','domain-number'])

    return df,model.components_

In [20]:
df_sl,components_pca_nsl=PCA_component_contribution(X_nsl_scaled,x_nsl)
df_nsl,components_pca_sl=PCA_component_contribution(X_sl_scaled,x_sl)

In [21]:
df_sl.head()

pca-component  domain-number
0           PC0            858
1           PC1            540
2           PC2              2
3           PC3           2233
4           PC4            341

## which are the shared domains between nSL and SL pairs , in the PCA reduced samples

In [22]:
df_shared_domains=pd.merge(df_sl,df_nsl,how='inner',on='domain-number')
domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_shared_domains['domain-description']=domains_description[df_shared_domains['domain-number']]


df_shared_domains

pca-component_x  domain-number pca-component_y  \
0              PC0            858             PC9   
1              PC8              3             PC7   
2             PC10           1253            PC83   
3             PC21            926            PC17   
4             PC23            463            PC18   
5             PC26            652            PC88   
6             PC49            215            PC19   
7             PC58             35           PC216   
8             PC63           1265           PC179   
9             PC68           1554           PC104   
10            PC69           2427            PC66   
11            PC69           2427            PC68   
12            PC73             61           PC110   
13            PC84             30           PC159   
14            PC86            369            PC93   
15            PC88            369            PC93   
16            PC90            271           PC129   
17            PC92           2244           PC135   
18           PC212           2244           PC135   
19            PC99           1743           PC228   
20           PC101           1743           PC228   
21           PC106            279            PC61   
22           PC112            685            PC90   
23           PC118            321           PC194   
24           PC120           1347           PC151   
25           PC124           1498           PC174   
26           PC124           1498           PC263   
27           PC125            381           PC167   
28           PC139           1236           PC253   
29           PC145              6           PC161   
30           PC145              6           PC163   
31           PC174           1733           PC186   
32           PC189            843           PC166   
33           PC199            310            PC50   
34           PC200           1571           PC239   
35           PC203           1962           PC178   
36           PC204           1570           PC124   
37           PC207           1570           PC124   
38           PC206              4            PC70   
39           PC211           2651           PC157   
40           PC213           1035           PC209   
41           PC222            647           PC190   
42           PC222            647           PC191   
43           PC235           1612           PC238   
44           PC245           2371            PC75   
45           PC253            316            PC44   
46           PC254           2485           PC140   
47           PC254           2485           PC141   

                                   domain-description  
0   F420_oxidored; Pyrroline-5-carboxylate reducta...  
1   2-Hacid_dh_C; D-isomer specific 2-hydroxyacid ...  
2               KA1; Kinase associated domain 1 (KA1)  
3                      Flp_C; Recombinase Flp protein  
4         Chitin_synth_1N; Chitin synthase N-terminal  
5         DUF1720; Domain of unknown function DUF1720  
6   Anticodon_1; Methionyl/Valyl/Leucyl/Isoleucyl-...  
7     AA_permease; Amino acid permease/ SLC12A domain  
8    Kei1; Protein of unknown function DUF1753, Golgi  
9              NAP; Nucleosome assembly protein (NAP)  
10                                         Scramblase  
11                                         Scramblase  
12                                               AGA2  
13             AAR2; A1 cistron-splicing factor, AAR2  
14                  CNH; Citron homology (CNH) domain  
15                  CNH; Citron homology (CNH) domain  
16                                               BLI1  
17               Ribosomal_S6e; Ribosomal protein S6e  
18               Ribosomal_S6e; Ribosomal protein S6e  
19                                              PHD_2  
20                                              PHD_2  
21                          BRAP2; BRCA1-associated 2  
22       DUF2722; Protein of unknown function DUF2722  
23                           CASP_C; CASP, C-terminal  
24   

In [23]:
print('The number of shared domains between SL and nSL pairs,after PCA, is = ',len(df_shared_domains), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_shared_domains)/(len(df_sl)+len(df_nsl)),'%')

The number of shared domains between SL and nSL pairs,after PCA, is =  48 out of 532 so the 9.022556390977444 %


## Domains exclusive to SL

In [24]:
df_sl_exclusive=pd.merge(df_sl,df_nsl,how='left',on='domain-number')

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_sl_exclusive['domain-description']=domains_description[df_sl_exclusive['domain-number']]
df_sl_exclusive[pd.isnull(df_sl_exclusive['pca-component_y'])]

pca-component_x  domain-number pca-component_y  \
1               PC1            540             NaN   
2               PC2              2             NaN   
3               PC3           2233             NaN   
4               PC4            341             NaN   
5               PC5           2610             NaN   
..              ...            ...             ...   
263           PC258           1418             NaN   
264           PC259           1458             NaN   
265           PC260            250             NaN   
266           PC261           2486             NaN   
267           PC262            818             NaN   

                                    domain-description  
1            Cytochrome_B; Cytochrome b/b6, N-terminal  
2    2-Hacid_dh; D-isomer specific 2-hydroxyacid de...  
3               Ribosomal_S27e; Ribosomal protein S27e  
4    CDP-OH_P_transf; CDP-alcohol phosphatidyltrans...  
5    TPK_B1_binding; Thiamin pyrophosphokinase, vit...  
..                                                 ...  
263                               Mak16; Mak16 protein  
264              Mem_trans; Membrane transport protein  
265                    B3_4; B3/B4 tRNA-binding domain  
266                       Sld5; GINS subunit, domain A  
267                                            ERGIC_N  

[220 rows x 4 columns]

In [25]:
print('The number of exclusive domains from SL pairs, after PCA, is = ',len(df_sl_exclusive), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_sl_exclusive)/(len(df_sl)+len(df_nsl)),'%')

The number of exclusive domains from SL pairs, after PCA, is =  268 out of 532 so the 50.37593984962406 %


In [26]:
df_nsl_exclusive=pd.merge(df_sl,df_nsl,how='right',on='domain-number')

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_nsl_exclusive['domain-description']=domains_description[df_nsl_exclusive['domain-number']]
df_nsl_exclusive[pd.isnull(df_nsl_exclusive['pca-component_x'])]

pca-component_x  domain-number pca-component_y  \
48              NaN           1028             PC0   
49              NaN            305             PC1   
50              NaN            268             PC2   
51              NaN            142             PC3   
52              NaN           1425             PC4   
..              ...            ...             ...   
268             NaN           1294           PC260   
269             NaN            302           PC261   
270             NaN            975           PC264   
271             NaN           1247           PC266   
272             NaN           1019           PC268   

                                    domain-description  
48                 GatB_Yqey; Asn/Gln amidotransferase  
49                     Brr6_like_C_C; Brl1/Brr6 domain  
50                   BHD_3; Rad4 beta-hairpin domain 3  
51       ATP_synt_H; ATPase, V0 complex, subunit e1/e2  
52            Mannosyl_trans; Mannosyltransferase, DXD  
..                                                 ...  
268                LIAS_N; Lipoyl synthase, N-terminal  
269                                  Brix; Brix domain  
270  GDPD; Glycerophosphodiester phosphodiesterase ...  
271  Ist1; Vacuolar protein sorting-associated prot...  
272      G_glu_transpept; Gamma-glutamyltranspeptidase  

[225 rows x 4 columns]

In [27]:
print('The number of exclusive domains from non SL pairs, after PCA, is = ',len(df_nsl_exclusive), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_nsl_exclusive)/(len(df_sl)+len(df_nsl)),'%')

The number of exclusive domains from non SL pairs, after PCA, is =  273 out of 532 so the 51.31578947368421 %


## Let see to the domains-number that get repeated after the PCA analysis , and selecting the most explanatory feature by its explained variance.

In [28]:
duplicated_features_sl=df_sl[df_sl.iloc[:,1].duplicated()]
repeated_features_sl=pd.DataFrame()

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

repeated_features_sl['domain-name']=domains_name[duplicated_features_sl.iloc[:,1]] 
repeated_features_sl['domain-description']=domains_description[duplicated_features_sl.iloc[:,1]]

duplicated_features_nsl=df_nsl[df_nsl.iloc[:,1].duplicated()]
repeated_features_nsl=pd.DataFrame()

repeated_features_nsl['domain-name']=domains_name[duplicated_features_nsl.iloc[:,1]] 
repeated_features_nsl['domain-description']=domains_description[duplicated_features_nsl.iloc[:,1]]

In [29]:
only_sl_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='left',on=['domain-name'])
data_only_sl = only_sl_pd[pd.isnull(only_sl_pd['domain-description_y'])]
data_only_sl['domain-description']=data_only_sl['domain-description_x']
data_only_sl.shape,df_sl_exclusive.shape

((37, 4), (268, 4))

In [30]:
only_nsl_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='right',on=['domain-name'])
data_only_nsl = only_nsl_pd[pd.isnull(only_nsl_pd['domain-description_x'])]
data_only_nsl

domain-name domain-description_x  \
0      PF01571                  NaN   
1      PF00790                  NaN   
2      PF06733                  NaN   
3      PF06733                  NaN   
4      PF08696                  NaN   
5      PF11916                  NaN   
6      PF00339                  NaN   
7      PF13401                  NaN   
8      PF00549                  NaN   
9      PF01267                  NaN   
10     PF13243                  NaN   
11     PF01302                  NaN   
12     PF04912                  NaN   
13     PF08801                  NaN   
14     PF09496                  NaN   
15     PF12352                  NaN   
16     PF04622                  NaN   
17     PF06413                  NaN   
18     PF00012                  NaN   
19     PF05046                  NaN   
20     PF00034                  NaN   
21     PF16965                  NaN   
22     PF01494                  NaN   
23     PF12550                  NaN   
24     PF01230                  NaN   
25     PF05032                  NaN   
26     PF01652                  NaN   
27     PF11935                  NaN   
28     PF03357                  NaN   
29     PF03357                  NaN   
30     PF03357                  NaN   
31     PF03357                  NaN   
32     PF04818                  NaN   
33     PF00155                  NaN   
34     PF00155                  NaN   
35     PF03194                  NaN   
36     PF00348                  NaN   
37     PF05024                  NaN   
38     PF00832                  NaN   
39     PF02301                  NaN   
40     PF11976                  NaN   
41     PF05160                  NaN   
42     PF08568                  NaN   
43     PF00022                  NaN   

                                 domain-description_y  
0             DUF2401; Cell wall protein YJL171C/Tos1  
1   COPI_assoc; Golgi apparatus membrane protein T...  
2     Ost5; Oligosaccharyltransferase complex subunit  
3     Ost5; Oligosaccharyltransferase complex subunit  
4                  RNA_pol_Rpb8; RNA polymerase, Rpb8  
5                                          Scramblase  
6   AdenylateSensor; AMPK, C-terminal adenylate se...  
7                     TPR_6; Tetratricopeptide repeat  
8   BCDHK_Adom3; Branched-chain alpha-ketoacid deh...  
9                DHC_N2; Dynein heavy chain, domain-2  
10   TOPRIM_C; C-terminal associated domain of TOPRIM  
11                               DJ-1_PfpI; DJ-1/PfpI  
12               Med4; Mediator complex, subunit Med4  
13                  RRM; RNA-binding domain, putative  
14  Rib_5-P_isom_A; Ribose 5-phosphate isomerase, ...  
15               Skp1_POZ; SKP1 component, POZ domain  
16   MOR2-PAG1_mid; Cell morphogenesis central region  
17  Nucleoporin_N; Nucleoporin, Nup133/Nup155-like...  
18                                     2OG-FeII_Oxy_4  
19  Mhr1; Mitochondrial homologous recombination p...  
20                                             AAA_11  
21  tRNA_int_end_N2; tRNA-splicing endonuclease, s...  
22        DUF1691; Domain of unknown function DUF1691  
23               Sortilin-Vps10; Sortilin, N-terminal  
24               DASH_Dam1; DASH complex subunit Dam1  
25                MgsA_C; MgsA AAA+ ATPase C-terminal  
26                DUF3453; Symplekin/Pta1, N-terminal  
27                                    Sds3; Sds3-like  
28                            HAND; ISWI, HAND domain  
29                            HAND; ISWI, HAND domain  
30                            HAND; ISWI, HAND domain  
31                            HAND; ISWI, HAND domain  
32        MatE; Multi antimicrobial extrusion protein  
33                 AIRS; PurM-like, N-terminal domain  
34                 AIRS; PurM-like, N-terminal domain  
35  Glycos_trans_3N; Glycosyl transferase family 3...  
36  Afi1; Arf3-interacting protein 1, N-terminal d...  
37  Mgm101p; Mitochondrial genome maintenance prot...  
38  CPDase; 2&apos;,3&apos;-cyclic-nucleotide 3&ap...  
39     

In [31]:
shared_domains_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='inner',on=['domain-name'])
shared_domains_pd


Empty DataFrame
Columns: [domain-name, domain-description_x, domain-description_y]
Index: []

In [32]:
a_sl=x_sl.iloc[:,np.sort(df_sl.iloc[:,1])]
a_sl.describe().loc['mean'].hist(bins=8),
a_sl.describe().loc['std'].hist(bins=8,alpha=0.4)
plt.xlim([0,0.2])

(0.0, 0.2)

In [33]:
a_nsl=x_nsl.iloc[:,np.sort(df_nsl.iloc[:,1])]
a_nsl.describe().loc['mean'].hist(bins=8),
a_nsl.describe().loc['std'].hist(bins=8,alpha=0.4)
plt.xlim([0,0.2])

(0.0, 0.2)

### Proof of concept with *BEM1*

- Gather the domains of SL and nSL pairs from BEM1
- Check if the domains for the SL and nSL pairs are inside the most important domains after PCA.

In [34]:
domains_bem1=data_domains[data_domains['name']=='BEM1']['domain-descrip']
sl_bem1=data_sl[data_sl['gene-query-name']=='BEM1']['gene-target-name']

In [35]:
sl_bem1

2846      BEM2
2847     CDC24
5004      MSB1
13777     SMI1
13778     BNI1
13779     BEM2
14385     SKN7
Name: gene-target-name, dtype: object

In [36]:
data_sl[data_sl['gene-query-name']=='BEM1']

gene-query gene-target gene-query-name gene-target-name  \
2846     YBR200W     YER155C            BEM1             BEM2   
2847     YBR200W     YAL041W            BEM1            CDC24   
5004     YBR200W     YOR188W            BEM1             MSB1   
13777    YBR200W     YGR229C            BEM1             SMI1   
13778    YBR200W     YNL271C            BEM1             BNI1   
13779    YBR200W     YER155C            BEM1             BEM2   
14385    YBR200W     YHR206W            BEM1             SKN7   

                                  gene-query-description  \
2846   SRO1|phosphatidylinositol-3-phosphate-binding ...   
2847   SRO1|phosphatidylinositol-3-phosphate-binding ...   
5004   SRO1|phosphatidylinositol-3-phosphate-binding ...   
13777  SRO1|phosphatidylinositol-3-phosphate-binding ...   
13778  SRO1|phosphatidylinositol-3-phosphate-binding ...   
13779  SRO1|phosphatidylinositol-3-phosphate-binding ...   
14385  SRO1|phosphatidylinositol-3-phosphate-binding ...   

                                 gene-target-description     interaction-type  \
2846                           IPL2|SUP9|TSL1|L000000168  Synthetic Lethality   
2847   CLS4|Rho family guanine nucleotide exchange fa...  Synthetic Lethality   
5004                                          L000001184  Synthetic Lethality   
13777                                    KNR4|L000000909  Synthetic Lethality   
13778                   PPF3|SHE5|formin BNI1|L000000190  Synthetic Lethality   
13779                          IPL2|SUP9|TSL1|L000000168  Synthetic Lethality   
14385  BRY1|POS9|kinase-regulated stress-responsive t...     Dosage Lethality   

            paper-source  
2846   Peterson J (1994)  
2847   Peterson J (1994)  
5004     Bender A (1991)  
13777   Gorelik M (2011)  
13778   Gorelik M (2011)  
13779   Gorelik M (2011)  
14385   Bouquin N (1999)